<a href="https://colab.research.google.com/github/Nandana230/major/blob/main/mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  transformers datasets sentencepiece
!pip install kaggle pymupdf spacy nltk
!python -m spacy download en_core_web_sm
!pip install --no-cache-dir torch==2.1.1 torchtext==0.16.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/usr/local/lib/python3.11/dist-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/usr/local/lib/python3.11/dist-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/usr/local/lib/python3.11/dist-packages/spacy/compat.py", line 4, in <module>
    from thinc.

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json manually


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nandanaapradee","key":"a02461f46fdb0bc49baa826fc7382cb1"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d Cornell-University/arxiv --unzip


Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv
License(s): CC0-1.0
arxiv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!kaggle datasets download -d Cornell-University/arxiv


Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv
License(s): CC0-1.0
arxiv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import os
print(os.listdir("arxiv"))


FileNotFoundError: [Errno 2] No such file or directory: 'arxiv'

In [ ]:
import zipfile
import os

if os.path.exists("arxiv.zip"):  # Ensure the file exists before extraction
    with zipfile.ZipFile("arxiv.zip", "r") as zip_ref:
        zip_ref.extractall("arxiv")  # Extract into 'arxiv' folder
    print("Extraction completed!")
else:
    print("Error: 'arxiv.zip' not found!")


Extraction completed!


In [ ]:
import pandas as pd

file_path = "arxiv/arxiv-metadata-oai-snapshot.json"

# Process dataset in chunks
chunk_size = 10000  # Load 10,000 rows at a time
data_chunks = pd.read_json(file_path, lines=True, chunksize=chunk_size)

# Initialize an empty list to store processed chunks
processed_data = []

for chunk in data_chunks:
    chunk = chunk[["title", "abstract"]]  # Load only required columns
    processed_data.append(chunk)  # Store the processed chunk

    # Display sample from the current chunk
    print(chunk.head())

# Concatenate all chunks into a single DataFrame if needed
df_full = pd.concat(processed_data, ignore_index=True)

print(f"Total records loaded: {len(df_full)}")




                                               title  \
0  Calculation of prompt diphoton production cros...   
1           Sparsity-certifying Graph Decompositions   
2  The evolution of the Earth-Moon system based o...   
3  A determinant of Stirling cycle numbers counts...   
4  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   

                                            abstract  
0    A fully differential calculation in perturba...  
1    We describe a new algorithm, the $(k,\ell)$-...  
2    The evolution of Earth-Moon system is descri...  
3    We show that a determinant of Stirling cycle...  
4    In this paper we show how to compute the $\L...  
                                                   title  \
10000               When Did Cosmic Acceleration Start ?   
10001    The Dirac system on the Anti-de Sitter Universe   
10002  Coupling of Optical Lumped Nanocircuit Element...   
10003  A model for learning to segment temporal seque...   
10004  Constructing a maximum uti

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text.strip()

# Example usage:
pdf_text = extract_text_from_pdf("sample.pdf")
print(pdf_text[:500])  # Preview first 500 characters


Received 12 August 2024, accepted 2 September 2024, date of publication 4 September 2024,
date of current version 12 September 2024.
Digital Object Identifier 10.1109/ACCESS.2024.3454543
Enhancing Cybersecurity With Artificial Immune
Systems and General Intelligence: A New Frontier
in Threat Detection and Response
OLUFUNSHO I. FALOWO
1, LILY EDINAM BOTSYOE1, KEHINDE KOSHOEDO
2,
AND MURAT OZER
1
1School of Information Technology, University of Cincinnati, Cincinnati, OH 45221, USA
2School of Buil


In [ ]:
import spacy
import re
import nltk
import pandas as pd
import os
from nltk.corpus import stopwords
from google.colab import drive  # Import Google Drive module

# Mount Google Drive
drive.mount('/content/drive')

# Define file paths in Google Drive
output_file = "/content/drive/My Drive/cleaned_arxiv.csv"  # Permanent storage
file_path = "arxiv/arxiv-metadata-oai-snapshot.json"

# Download stopwords
nltk.download('stopwords')

# Load spaCy model & stopwords
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'\[\d+\]', '', text)  # Remove citations [1], [2]
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    doc = nlp(text)
    return " ".join([token.text for token in doc if token.is_alpha and token.text.lower() not in stop_words])

# Define chunk size
chunk_size = 10000
data_chunks = pd.read_json(file_path, lines=True, chunksize=chunk_size)

# Check if a partially saved file exists
if os.path.exists(output_file):
    processed_chunks = sum(1 for _ in open(output_file)) - 1  # Count existing lines
else:
    processed_chunks = 0

print(f"Resuming from chunk {processed_chunks}")

# Process dataset in chunks & save after each chunk
for i, chunk in enumerate(data_chunks):
    if i < processed_chunks:  # Skip already processed chunks
        continue

    chunk["clean_abstract"] = chunk["abstract"].apply(clean_text)

    # Save each chunk to Google Drive (append mode)
    chunk.to_csv(output_file, mode="a", header=not os.path.exists(output_file), index=False)

    print(f"Processed & saved chunk {i+1}")

print(f"Processing completed! Cleaned data saved in {output_file}")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Resuming from chunk 8307525
Processing completed! Cleaned data saved in /content/drive/My Drive/cleaned_arxiv.csv


In [ ]:
import pandas as pd
import os

output_path = "/content/drive/My Drive/processed_arxiv.json"
file_path = "arxiv/arxiv-metadata-oai-snapshot.json"

# Define chunk size
chunk_size = 10000

# Check if a partially saved file exists and count processed chunks
if os.path.exists(output_path):
    processed_chunks = sum(1 for _ in open(output_path))  # Count lines (each JSON object is a line)
else:
    processed_chunks = 0

print(f"Resuming from chunk {processed_chunks}")

# Re-initialize data_chunks
data_chunks = pd.read_json(file_path, lines=True, chunksize=chunk_size)

# Process remaining chunks
for i, chunk in enumerate(data_chunks):
    if i < processed_chunks:  # Skip already processed chunks
        continue

    chunk["clean_abstract"] = chunk["abstract"].apply(clean_text)
    chunk.to_json(output_path, orient="records", lines=True, mode="a")  # Append mode
    print(f"Saved chunk {i+1} to disk")




Resuming from chunk 310000


In [ ]:
import torch
import pandas as pd
import os
import pickle
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import torch.nn as nn

In [ ]:
df_cleaned = pd.read_json(output_path, lines=True)
print(f"Loaded {len(df_cleaned)} records from processed dataset")


Loaded 310000 records from processed dataset


Loaded 31000 records for training


In [ ]:
!pip uninstall torchtext torch -y
!pip install torch torchtext --no-cache-dir

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 150.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 116.1 MB/s eta 0:00:00


In [ ]:
import torch.nn.functional

In [ ]:
!pip install --no-cache-dir torch==2.1.1 torchtext==0.16.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.9/184.9 MB 192.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 178.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 195.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 210.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 281.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 290.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 210.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 285.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 198.8 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of torchvision to deter

In [ ]:
!pip show torchtext

Name: torchtext
Version: 0.16.1+cpu
Summary: Text utilities, models, transforms, and datasets for PyTorch.
Home-page: https://github.com/pytorch/text
Author: PyTorch Text Team
Author-email: packages@pytorch.org
License: BSD
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy, requests, torch, torchdata, tqdm
Required-by: 


In [ ]:
!pip install torchtext

In [ ]:
 # Extract cleaned abstracts
import pandas as pd
output_path = "/content/drive/My Drive/processed_arxiv.json"
df_subset = pd.read_json(output_path, lines=True)
papers_cleaned = df_subset["clean_abstract"].tolist()
summaries_cleaned = df_subset["clean_abstract"].tolist()  # Replace with actual summaries if available

from collections import Counter
# Tokenizer

from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer("basic_english")

# Create vocabulary
word_counts = Counter()
for text in papers_cleaned + summaries_cleaned:
    word_counts.update(tokenizer(text))

vocab = {word: idx + 1 for idx, (word, _) in enumerate(word_counts.most_common())}
vocab["<pad>"] = 0  # Add padding token

class SummarizationDataset(Dataset):
    def __init__(self, texts, summaries, vocab):
        self.texts = [torch.tensor([vocab.get(word, 0) for word in tokenizer(text)]) for text in texts]
        self.summaries = [torch.tensor([vocab.get(word, 0) for word in tokenizer(summary)]) for summary in summaries]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.summaries[idx]

# **Padding function for DataLoader**
def collate_fn(batch):
    text_batch, summary_batch = zip(*batch)

    # Ensure tensors are properly detached before processing
    text_batch = [seq.clone().detach() if isinstance(seq, torch.Tensor) else torch.tensor(seq) for seq in text_batch]
    summary_batch = [seq.clone().detach() if isinstance(seq, torch.Tensor) else torch.tensor(seq) for seq in summary_batch]

    # Pad sequences
    text_batch = pad_sequence(text_batch, batch_first=True, padding_value=0)
    summary_batch = pad_sequence(summary_batch, batch_first=True, padding_value=0)

    return text_batch, summary_batch



# Create dataset and dataloader
dataset = SummarizationDataset(papers_cleaned, summaries_cleaned, vocab)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Extract cleaned abstracts
import pandas as pd
output_path = "/content/drive/My Drive/processed_arxiv.json"
df_subset = pd.read_json(output_path, lines=True) # Load your processed dataset
papers_cleaned = df_subset["clean_abstract"].tolist()
summaries_cleaned = df_subset["clean_abstract"].tolist()  # Replace with actual summaries if available

from collections import Counter
# Tokenizer

from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer("basic_english")

vocab_path = "/content/drive/MyDrive/vocab.pkl"


# Create vocabulary or load existing one
if os.path.exists(vocab_path):
    print("Loading existing vocabulary...")
    with open(vocab_path, "rb") as f:
        vocab = pickle.load(f)
else:
    print("Creating new vocabulary...")
    word_counts = Counter()
    for text in papers_cleaned + summaries_cleaned:
        word_counts.update(tokenizer(text))

    vocab = {word: idx + 1 for idx, (word, _) in enumerate(word_counts.most_common())}
    vocab["<pad>"] = 0  # Add padding token

    with open(vocab_path, "wb") as f:
        pickle.dump(vocab, f)
    print(f"Vocabulary saved to {vocab_path}") # This line was incorrectly indented. Fixed it to align with 'with open' statement

class SummarizationDataset(Dataset):
    def __init__(self, texts, summaries, vocab):
        self.texts = [torch.tensor([vocab.get(word, 0) for word in tokenizer(text)]) for text in texts]
        self.summaries = [torch.tensor([vocab.get(word, 0) for word in tokenizer(summary)]) for summary in summaries]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.summaries[idx]

# **Padding function for DataLoader**
def collate_fn(batch):
    text_batch, summary_batch = zip(*batch)

    # Ensure tensors are properly detached before processing
    text_batch = [seq.clone().detach() if isinstance(seq, torch.Tensor) else torch.tensor(seq) for seq in text_batch]
    summary_batch = [seq.clone().detach() if isinstance(seq, torch.Tensor) else torch.tensor(seq) for seq in summary_batch]


    # Pad sequences
    text_batch = pad_sequence(text_batch, batch_first=True, padding_value=0)
    summary_batch = pad_sequence(summary_batch, batch_first=True, padding_value=0)

    return text_batch, summary_batch



# Create dataset and dataloader
dataset = SummarizationDataset(papers_cleaned, summaries_cleaned, vocab)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Creating new vocabulary...
Vocabulary saved to /content/drive/MyDrive/vocab.pkl


In [ ]:
!pip install numpy==1.26.4

In [ ]:
class TransformerSummarizer(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, num_layers, hidden_size):
        super(TransformerSummarizer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.transformer = nn.Transformer(d_model=embed_size, nhead=num_heads, num_encoder_layers=num_layers)
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        output = self.transformer(embedded, embedded)
        return F.log_softmax(self.fc(output), dim=-1)

In [ ]:
model_save_path = "/content/drive/MyDrive/transformer_checkpoint.pth"

import torch.nn.functional as F
# Initialize model
model = TransformerSummarizer(vocab_size=len(vocab), embed_size=256, num_heads=8, num_layers=6, hidden_size=512)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding tokens

# **Load checkpoint if exists**
def load_checkpoint():
    if os.path.exists(model_save_path):
        print("Loading existing model for incremental training...")
        checkpoint = torch.load(model_save_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        batch_count = checkpoint['batch_count']
        print(f"Resuming training from Epoch {start_epoch}, Batch {batch_count}")
        return start_epoch, batch_count
    else:
        print("No checkpoint found. Starting fresh.")
        return 0, 0  # Start from scratch

# **Train model with checkpointing**
def train_model(start_epoch=0, batch_count=0):
    model.train()
    checkpoint_interval = 500  # Save every 500 batches

    for epoch in range(start_epoch, 5):  # Reduce epochs per subset
        total_loss = 0

        for batch_idx, (text, summary) in enumerate(dataloader, start=batch_count):
            text, summary = text.to(device), summary.to(device)

            optimizer.zero_grad()
            output = model(text)

            loss = criterion(output.view(-1, output.shape[-1]), summary.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            #  Save checkpoint every 500 batches
            if batch_idx % checkpoint_interval == 0:
                torch.save({
                    'epoch': epoch,
                    'batch_count': batch_idx,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, model_save_path)
                print(f"Checkpoint saved at batch {batch_idx} (Epoch {epoch + 1})")

        print(f"Epoch {epoch + 1}, Average Loss: {total_loss / len(dataloader):.4f}")


        torch.save({
            'epoch': epoch,
            'batch_count': 0,  # Reset batch count for new epoch
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, model_save_path)
        print(f"Checkpoint saved at {model_save_path} (End of Epoch {epoch + 1})")


start_epoch, batch_count = load_checkpoint()
train_model(start_epoch=start_epoch, batch_count=batch_count)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Loading existing model for incremental training...


RuntimeError: Error(s) in loading state_dict for TransformerSummarizer:
	size mismatch for embedding.weight: copying a param with shape torch.Size([47620, 256]) from checkpoint, the shape in current model is torch.Size([135222, 256]).
	size mismatch for fc.weight: copying a param with shape torch.Size([47620, 256]) from checkpoint, the shape in current model is torch.Size([135222, 256]).
	size mismatch for fc.bias: copying a param with shape torch.Size([47620]) from checkpoint, the shape in current model is torch.Size([135222]).

In [ ]:
model_save_path = "/content/drive/MyDrive/transformer_checkpoint.pth"

import torch.nn.functional as F
# Get vocab_size from the saved checkpoint instead of len(vocab)
checkpoint = torch.load(model_save_path, map_location=device)
vocab_size = checkpoint['model_state_dict']['embedding.weight'].shape[0]

# Initialize model with the correct vocab_size
model = TransformerSummarizer(vocab_size=vocab_size, embed_size=256, num_heads=8, num_layers=6, hidden_size=512)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding tokens

# **Load checkpoint if exists**
def load_checkpoint():
    if os.path.exists(model_save_path):
        print("Loading existing model for incremental training...")
        checkpoint = torch.load(model_save_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        batch_count = checkpoint['batch_count']
        print(f"Resuming training from Epoch {start_epoch}, Batch {batch_count}")
        return start_epoch, batch_count
    else:
        print("No checkpoint found. Starting fresh.")
        return 0, 0  # Start from scratch

# **Train model with checkpointing**
def train_model(start_epoch=0, batch_count=0):
    model.train()
    checkpoint_interval = 500  # Save every 500 batches

    for epoch in range(start_epoch, 5):  # Reduce epochs per subset
        total_loss = 0

        for batch_idx, (text, summary) in enumerate(dataloader, start=batch_count):
            text, summary = text.to(device), summary.to(device)

            optimizer.zero_grad()
            output = model(text)

            loss = criterion(output.view(-1, output.shape[-1]), summary.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            #  Save checkpoint every 500 batches
            if batch_idx % checkpoint_interval == 0:
                torch.save({
                    'epoch': epoch,
                    'batch_count': batch_idx,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, model_save_path)
                print(f"Checkpoint saved at batch {batch_idx} (Epoch {epoch + 1})")

        print(f"Epoch {epoch + 1}, Average Loss: {total_loss / len(dataloader):.4f}")

        #  Save checkpoint at the end of each epoch
        torch.save({
            'epoch': epoch,
            'batch_count': 0,  # Reset batch count for new epoch
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, model_save_path)
        print(f"Checkpoint saved at {model_save_path} (End of Epoch {epoch + 1})")

# **Load checkpoint & Resume Training**
start_epoch, batch_count = load_checkpoint()
train_model(start_epoch=start_epoch, batch_count=batch_count)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Loading existing model for incremental training...
Resuming training from Epoch 5, Batch 9500


In [ ]:
def generate_summary(text, length="short"):
    length_map = {"short": 100, "medium": 250, "long": 500}
    max_length = length_map.get(length, 150)

    input_seq = torch.tensor([vocab[word] for word in tokenizer(text)]).unsqueeze(0)
    output = model(input_seq)
    summary_ids = torch.argmax(output, dim=-1)[0][:max_length]
    summary = " ".join([rev_vocab[idx] for idx in summary_ids])
    return summary


In [ ]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim

# Mount Google Drive
drive.mount('/content/drive')

# Define model path
model_save_path = "/content/drive/MyDrive/transformer_checkpoint.pth"

# Load the checkpoint
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(model_save_path, map_location=device)

# Extract vocab size
vocab_size = checkpoint['model_state_dict']['embedding.weight'].shape[0]

# Define the TransformerSummarizer model
class TransformerSummarizer(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, num_layers, hidden_size):
        super(TransformerSummarizer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.transformer = nn.Transformer(d_model=embed_size, nhead=num_heads, num_encoder_layers=num_layers)
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        output = self.transformer(embedded, embedded)
        return torch.log_softmax(self.fc(output), dim=-1)

# Load model
model = TransformerSummarizer(vocab_size=vocab_size, embed_size=256, num_heads=8, num_layers=6, hidden_size=512)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()  # Set model to evaluation mode


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TransformerSummarizer(
  (embedding): Embedding(47620, 256)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLay

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = [page.get_text("text") for page in doc]
    return "\n".join(text)  # Join all pages

# Example usage
pdf_path = "sample.pdf"
raw_text = extract_text_from_pdf(pdf_path)
print(raw_text[:1000])  # Print first 1000 characters



Received 12 August 2024, accepted 2 September 2024, date of publication 4 September 2024,
date of current version 12 September 2024.
Digital Object Identifier 10.1109/ACCESS.2024.3454543
Enhancing Cybersecurity With Artificial Immune
Systems and General Intelligence: A New Frontier
in Threat Detection and Response
OLUFUNSHO I. FALOWO
1, LILY EDINAM BOTSYOE1, KEHINDE KOSHOEDO
2,
AND MURAT OZER
1
1School of Information Technology, University of Cincinnati, Cincinnati, OH 45221, USA
2School of Built Environment, Faculty of Technology, Environment and Design, Oxford Brookes University, OX3 0BP Oxford, U.K.
Corresponding author: Olufunsho I. Falowo (falowooi@mail.uc.edu)
ABSTRACT
This study explores how integrating Artificial General Intelligence (AGI) with Artificial
Immune Systems (AIS) could potentialy enhance the efficiency of Security Operations Centers (SOCs).
By employing a hypothetical case study and mathematical models, this research compares AGI-driven
AIS with traditional AI-driv

In [ ]:
import re

def split_into_sections(text):
    # Define section headers (Modify based on paper structure)
    section_headers = [
        "Abstract", "Introduction", "Methodology", "Experiments", "Results",
        "Discussion", "Conclusion", "References"
    ]

    sections = {}
    current_section = None
    for line in text.split("\n"):
        if line.strip() in section_headers:
            current_section = line.strip()
            sections[current_section] = []
        elif current_section:
            sections[current_section].append(line.strip())

    # Convert section lists to text
    for key in sections:
        sections[key] = " ".join(sections[key])

    return sections

print("Extracted Sections:", sections.keys())  # Print section names


# Example usage
sections = split_into_sections(raw_text)
for section, content in sections.items():
    print(f"--- {section} ---\n{content[:500]}...\n")  # Print first 500 chars per section

for section, content in sections.items():
    print(f"Section: {section}, Content (First 200 chars): {content[:200]}")



Extracted Sections: dict_keys([])


In [ ]:
from collections import Counter
from torchtext.data.utils import get_tokenizer
import torch

tokenizer = get_tokenizer("basic_english")

# Reuse existing vocab from training
word_to_idx = checkpoint['model_state_dict']['embedding.weight'].shape[0]

def encode_text(text, vocab):
    return torch.tensor([vocab.get(word, 0) for word in tokenizer(text)], dtype=torch.long)

# Encode sections
encoded_sections = {section: encode_text(content, word_to_idx) for section, content in sections.items()}


In [ ]:
def generate_summary(input_tensor, model, max_len=100):
    model.eval()
    input_tensor = input_tensor.unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        output = model(input_tensor)
    output_indices = torch.argmax(output, dim=-1).squeeze().tolist()

    # Convert indices back to words
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    summary = " ".join([idx_to_word.get(idx, "<unk>") for idx in output_indices][:max_len])

    return summary

# Generate summaries
summaries = {section: generate_summary(encoded_text, model) for section, encoded_text in encoded_sections.items()}

# Print summaries
for section, summary in summaries.items():
    print(f"--- Summary of {section} ---\n{summary}\n")
